In [0]:
import http.server
import socketserver
import datetime
import sqlite3




class RequestHandler(http.server.SimpleHTTPRequestHandler): # définition du nouveau handler, classe dérivée de http.server.SimpleHTTPRequestHandler

    static_dir = '/client' # sous-répertoire racine des documents statiques
    
    def do_GET(self): # on surcharge la méthode qui traite les requêtes GET       
        if self.path[0:5]!='/time' and self.path[0:10]!='/countries' :
            self.path = self.static_dir + self.path  # on modifie le chemin d'accès en insérant un répertoire préfixe 
            http.server.SimpleHTTPRequestHandler.do_GET(self) # on traite la requête via la classe parent
           
        
        if self.path[0:5]=='/time':  #si le chemin se termine par '/time'
            affichage_date=datetime.datetime.today().ctime()           
            response='<!DOCTYPE html><title>hello</title>' + \
"<meta charset='utf-8'><p>Voici l'heure du serveur : {} GMT </p>"\
.format(affichage_date)
            self.send(response)      
            
        
        if self.path[0:10]=='/countries':
            conn=sqlite3.connect('pays.sqlite')
            c=conn.cursor()
            c.execute("SELECT name FROM countries")
            a=c.fetchall()
            liste_pays=[a[i][0] for i in range(len(a))]  #stocke la liste des tables de la database (strings)
            
            
            if  len(liste_pays)==0:
                response='<!DOCTYPE html><title>hello</title>' + \
"<meta charset='utf-8'><p>Pas de pays dans la database  </p>"
                self.send(response)              
                
                
            else:               
                        
                response='<!DOCTYPE html><title>hello</title>' + \
"<meta charset='utf-8'><p>Liste des pays :  </p>" 
                
                
                for i in range(len(liste_pays)):
                    response=response + \
"<meta charset='utf-8'><p>"+"["+str(i+1)+"] - "+liste_pays[i]+"</p>"                       
                self.send(response)

            
            
            
            
            
            
            
            
            

    def send(self,body,headers=[]):     
        
        encoded = bytes(body, 'UTF-8')  # on encode la chaine de caractères à envoyer, body est une string           
        self.send_response(200)   # on envoie la ligne de statut       
        [self.send_header(*t) for t in headers] # on envoie les lignes d'entête et la ligne vide
        self.send_header('Content-Length',int(len(encoded)))
        self.end_headers()        
        self.wfile.write(encoded) # on envoie le corps de la réponse

    
httpd = socketserver.TCPServer(("", 8080), RequestHandler)# instanciation et lancement du serveur
httpd.serve_forever()